In [3]:
from Model.model import ModelSTGCN
import torch
from torch.utils.data import DataLoader,Dataset
import pandas as pds
import numpy as np
import torch.nn as nn

c:\Users\PhuTuan\anaconda3\envs\pythonProject\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [49]:
class ActionDataset(Dataset):
    def __init__(self,Transform=None):
        super().__init__()
        # assert all(param is not None for param in [Data,label]),"Data and label must be give in"
        self.transform = Transform
        self.lst=[]
        self.label=[]
        # self.append(Data,label)

    def __getitem__(self, index):
        sample = self.lst[index],self.label[index]
        if self.transform:
            sample=self.transform(sample)
        return sample
    
    def __len__(self):
        return self.leng
    
    def SetTrans(self,Transform):
        self.transform=Transform
    
    def append(self,Data):
        # super().__init__()
        # assert all(param is not None for param in [Data,label]),"Data and label must be give in"
        # assert label==None,"Label must be give"
        kpscore = np.expand_dims(Data['kp_score'][0],axis=2)
        kp = Data['kp'][0]
        data = np.concatenate((kp,kpscore),axis=2)
        data = np.expand_dims(data,axis=0)
        label = Data['label']
        self.lst.append(data)
        self.label.append(label)
        self.leng=len(self.lst)  

class ToTensor():
    def __call__(self,sample):
        data,label=sample
        return torch.from_numpy(data),torch.tensor(label)

In [5]:
def train_val(model,dataset,lr,epochs,device='cuda'):
    device=torch.device(device)
    model.to(device)
    optim = torch.optim.adam(model.parameter(),lr)
    criteria = nn.CrossEntropyLoss()
    dataloader=DataLoader(dataset=dataset,shuffle=True)
    # n_sample=len(dataset)
    epochs=10
    for epoch in range(epochs-8):
        for i,(data,label) in enumerate (dataloader):
            data = data.to(device)
            label = label.to(device)
            predict = model(data)
            loss = criteria(label,predict)
            optim.zero_grad()
            loss.backward()
            optim.step()
    print(f'Epoch: {epoch}, lr = {lr:.5d}, Loss_train: {loss:.5d}, Loss_val:')

In [ ]:
# kp=datalst[0]['kp'][0]
# kp_score=datalst[0]['kp_score'][0]
# kp_score=np.expand_dims(kp_score,axis=2)
# a=np.concatenate((kp,kp_score),axis=2)
# a.shape


In [22]:
# batch_size=4
# datalst=pds.read_pickle('train.pkl')
# actDataset=ActionDataset()
# actDataset.append(datalst[0])
# actDataset.SetTrans(ToTensor())
# data=actDataset[0]

In [50]:
actDataset = ActionDataset(ToTensor())
datalst = pds.read_pickle('train.pkl')
leng = len(datalst)
for data in datalst :
    actDataset.append(data)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [57]:
# for data in datalst:
#     # data_shape = data['kp_score'][0].shape
#     # print(f'data kp shape: {data_shape}')
#     actDataset.append(data)
# # data=datalst[0]
data['kp_score'].shape
datalst[47]

{'kp': array([], shape=(0, 50, 17, 2), dtype=float32),
 'kp_score': array([], shape=(0, 50, 17), dtype=float32),
 'frame_dir': 'VIDEO1_5',
 'img_shape': (1280, 722),
 'original_shape': (1280, 722),
 'total_frames': 50,
 'label': 2}

In [18]:
a=np.expand_dims(data['kp_score'][0],axis=2)
a.shape

(292, 17, 1)

In [ ]:
actDataset.append()

In [ ]:
datalst[0]['kp'].shape

In [ ]:
batch_size=4
datalst=pds.read_pickle('train.pkl')
actDataset=ActionDataset(datalst[0])
dataloader=DataLoader(dataset=actDataset,shuffle=True)
n_sample=len(actDataset)
epochs=10
n_iter=round(n_sample/batch_size)
for epoch in range(epochs-8):
    for i,data in enumerate (dataloader):
        if (i+1)% (batch_size)==0:
            print(f'Epoch: {epoch}, Step: {i+1}/{n_iter}, Input shap: {data.shape}')

In [ ]:
import math
math.ceil(len(actDataset)/batch_size)

In [ ]:
batch_size=4
datalst=pds.read_pickle('train.pkl')
actDataset=ActionDataset(datalst[0])
dataloader=DataLoader(dataset=actDataset,batch_size=batch_size,shuffle=True)
n_sample=len(actDataset)
n_iter=round(n_sample/batch_size)
model = ModelSTGCN(3,5)
lr = 1e-3
optim = torch.optim.Adam(model.parameters(),lr)
lr_schedule = torch.optim.lr_scheduler.MultiStepLR(optim,[15,30],gamma=0.1)
epochs = 10
for epoch in range(epochs):
    for i,data in enumerate (dataloader):
        if (i+1)% (batch_size)==0:
            print(f'Epoch: {epoch}, Step: {i+1}/{n_iter}')